In [3]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('../data/ecom_data.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Minimum OrderValue Select'),
        # Add a slider input
        dcc.Slider(id='value_slider',
            # Set the min and max of the slider
            min=ecom_sales['OrderValue'].min(),
            max=ecom_sales['OrderValue'].max(),
            # Start the slider at 0
            value=0,
            # Increment the slider by 50 each notch
            step=50,
            vertical=False)
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            dcc.Graph(id='sales_cat'),
            html.H2('Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    # Set the input and output of the callback to link the dropdown to the graph
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='value_slider', component_property='value')
)

def update_plot(min_val):
    
    sales = ecom_sales.copy(deep=True)

    if min_val:
        sales = sales[sales['OrderValue'] >= min_val]

    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].size().reset_index(name='Total Sales Volume')

    bar_fig_major_cat = px.bar(
        title=f'Sales with order value: {min_val}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales Volume', y='Major Category')

    return bar_fig_major_cat

# Set the app to run in development mode
if __name__ == '__main__':
    app.run_server(debug=True, port=8054)